<a href="https://colab.research.google.com/github/xoxominji/22-1-ESAA-Practice/blob/main/0411_%EB%B0%95%EB%AF%BC%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [개념정리 및 필사]

## 핸즈온 Chapter 9. 비지도학습

이상치 탐지: '정상' 데이터가 어떻게 보이느닞 학습 -> 비정상 샘플 갑지 사용

밀도 추정: 데이터셋 생성 확률 과정의 pdf를 추정. 

### 9.1 군집

: 비슷한 샘플을 구별해 하나의 클러스터(cluster)로 모으기

<애플리케이션>
- 고객 분류: 고객 그룹마다 제품 추천/마케팅 전략 적용 가능. 추천 시스템 구축에 도움
- 데이터 분석: 각 클러스터 별로 따로 분석시 도움이 됨
- 차원 축소 기법: 각 클러스터에 대한 샘플의 친화성(샘플이 클러스터에 얼마나 잘 맞는가?) 측정
- 이상치 탐지: 친화성이 낮은 샘플 감지
- 준지도 학습: 레이블된 샘플이 적을시 동일한 클러스터에 있는 모든 샘플에 레이블 전파
- 검색 엔진: 제시된 이미지와 비슷한 이미지 찾기
- 이미지 분할: 색 기반 픽셀 클러스터 모은 후 각 픽셀의 색을 해당 클러스터의 평균 색으로 바꿔 색상의 종류를 크게 줄임

#### 9.1.1 k-평균



```
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=#, init=np.array([], [], ...), n_init=#)
```



In [ ]:
# 예제 데이터 생성
import numpy as np
from sklearn.datasets import make_blobs

blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [ ]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(X)
y_pred

array([4, 0, 1, ..., 2, 1, 0], dtype=int32)

In [ ]:
y_pred is kmeans.labels_

True

In [ ]:
kmeans.cluster_centers_

array([[-2.80389616,  1.80117999],
       [ 0.20876306,  2.25551336],
       [-2.79290307,  2.79641063],
       [-1.46679593,  2.28585348],
       [-2.80037642,  1.30082566]])

In [ ]:
#새로운 샘플의 레이블 예측
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.predict(X_new)

array([1, 1, 2, 2], dtype=int32)

단점: 클러스터의 크기가 많이 다르면 작동 안됨

- 하드 군집: 각 샘플에 대해 가장 가까운 군집 선택
- 소프트 군집: 클러스터마다 샘플에 점수(각 샘플에서 모든 센트로이드까지 거리, 가우시안 방사기저 함수, 유사도 점수, 친화성 점수 등) 부여
  - transform() 메서드 활용
  - 측정 값이 다른 분류기 등에 활용될 수 있음

In [ ]:
kmeans.transform(X_new)

array([[2.81093633, 0.32995317, 2.9042344 , 1.49439034, 2.88633901],
       [5.80730058, 2.80290755, 5.84739223, 4.4759332 , 5.84236351],
       [1.21475352, 3.29399768, 0.29040966, 1.69136631, 1.71086031],
       [0.72581411, 3.21806371, 0.36159148, 1.54808703, 1.21567622]])

##### k-평균 알고리즘

- 먼저 k개의 센트로이드 무작위 선택
- 센트로이드의 위치가 더 이상 변하지 않을 때까지 아래 과정 반복
  - 각 샘플을 가장 가까운 센트로이드에 할당
  - 센트로이드에 할당된 샘플들의 평균으로 센트로이드 업데이트

##### 센트로이드 초기화 방법
- init 매개변수에 센트로이드 리스트 담은 넘파이 배열 지정 (n_init=1)
- n_init 매개변수로 랜덤 초기화 횟수 조절


<성능 지표>
- inertia_ 인스턴스 변수: 이터셔(센트로이드 사이의 평균 제곱 거리)
- score 메서드

In [ ]:
kmeans.inertia_

211.5985372581684

In [ ]:
kmeans.score(X)

-211.59853725816836

##### k-평균 속도 개선과 미니배치 k-평균

: 불필요한 거리계산 제외

```
from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans = MiniBatchKMeans(n_clusters=#)
```

단점: 일반 k-평균보다 빠르지만 이너셔가 더 나쁨(클러스터 개수 많을 때)


In [ ]:
from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans = MiniBatchKMeans(n_clusters=5, random_state=42)
minibatch_kmeans.fit(X)

MiniBatchKMeans(n_clusters=5, random_state=42)

##### 최적의 클러스터 개수 찾기

: 실루엣 점수(실루엣 계수의 평균) 참고

: -1(샘플이 잘못된 클러스터에 할당) ~ 0(클러스터 경계에 위치) ~ +1(잘 속해있음)

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

0.655517642572828

#### 9.1.2 k-평균의 한계

장점: 속도가 빠르고 확장에 용이

단점: 알고리즘 여러 번 실행, 클러스터 개수 지정, 밀집도가 서로 다르거나 클러스터가 원형이 아닐 경우 잘 작동못함

#### 9.1.3 군집을 사용한 이미지 분할

- 이미지 분할: 이미지를 세그먼트로 여러 개로 분할

- 시맨틱 분할: 동일한 종류의 물체에 속한 모드 픽셀은 같은 세그먼트에 할당

- 색상 분할: 동일한 색상을 가진 픽셀을 같은 세그먼트에 할당

#### 9.1.4 군집을 사용한 전처리

: 파이프라인을 만들어 먼저 훈련 세트를 n가의 클러스터로 모은 후 이미지를 클러스터까지 거리로 바꿈 -> 로지스틱 회귀 모델 적용

In [ ]:
from sklearn.datasets import load_digits
X_digits, y_digits = load_digits(return_X_y=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=42)

In [ ]:
log_reg.score(X_test, y_test)

0.9733333333333334

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50, random_state=42)),
    ("log_reg", LogisticRegression(random_state=42)),
])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('kmeans', KMeans(n_clusters=50, random_state=42)),
                ('log_reg', LogisticRegression(random_state=42))])

In [ ]:
pipeline.score(X_test, y_test)

0.9688888888888889

GridSearchCV를 사용해 최적에 클러스터 개수 찾을 수 있음

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

In [ ]:
grid_clf.best_params_
grid_clf.score(X_test, y_test)

#### 9.1.5 군집을 사용한 준지도 학습

In [ ]:
n_labeled = 50
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", random_state=42)
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])
log_reg.score(X_test, y_test)

0.8333333333333334

각 클러스에서 센트로이드에 가장 가까운 이미지(대표 이미지)를 활용해 수동으로 레이블을 할당함으로써 성능 개선 가능

In [ ]:
k = 50
kmeans = KMeans(n_clusters=k, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

In [ ]:
y_representative_digits = np.array([
    0, 1, 3, 2, 7, 6, 4, 6, 9, 5,
    1, 2, 9, 5, 2, 7, 8, 1, 8, 6,
    3, 1, 5, 4, 5, 4, 0, 3, 2, 6,
    1, 7, 7, 9, 1, 8, 6, 5, 4, 8,
    5, 3, 3, 6, 7, 9, 7, 8, 4, 9])

In [ ]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

동일한 클러스터에 있는 모든 샘플에 레이블 전파(레이블 전파) 후 학습시켜 성능 더 개선 가능

In [ ]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [ ]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test, y_test)

클러스터 경계에 가깝게 위치한 샘플의 레이블이 잘못될 가능성 존재

센트로이드와 가까운 샘플의 20%에만 레이블 전파

In [ ]:
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1
    
partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [ ]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)

#### 9.1.6 DBSCAN

- 알고리즘이 각 샘플에서 작은 거리인 입실론 내에 샘플이 몇 개 놓여 있는지 세기 (e-neighborhood)
- e-이웃 내에 적어도 min_sampels개 샘플이 있다면 이를 핵심 샘플로 간주
- 핵심 샘플의 이웃에 있는 모든 샘플은 동일한 클러스터에 소속. 핵심 샘플의 이웃의 이웃은 계속해서 하나의 클러스터 형성
- 핵심 샘플, 이웃도 아닌 샘플은 이상치 처리



```
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=%, min_samples=#) #하이퍼파라미터는 2개뿐
dbscan.fit_predict() #predict() 메서드 없음
```

  - labels_ 인스턴스 변수: 클러스터 인덱스 (-1은 이상치)
  - core_sample_indices_인스턴스 변수: 핵심 샘플의 인텍스
  - components_ 인스턴스 변수: 핵심 샘플 자체


<장점> 클러스터의 모양과 개수에 상관없이 감지할 수 있음, 이상치에 안정적, 하이퍼파라미터 2개

<단점> 클러스터 간의 밀집도가 크게 다르면 모든 클러스터를 올바르게 잡아내는 것은 불가능

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=1000, noise=0.05, random_state=42)

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [ ]:
dbscan.labels_[:10]

array([ 0,  2, -1, -1,  1,  0,  0,  0,  2,  5])

In [ ]:
len(dbscan.core_sample_indices_)

808

In [ ]:
dbscan.core_sample_indices_[:10]

array([ 0,  4,  5,  6,  7,  8, 10, 11, 12, 13])

In [ ]:
dbscan.components_[:3]

array([[-0.02137124,  0.40618608],
       [-0.84192557,  0.53058695],
       [ 0.58930337, -0.32137599]])

#### 9.1.7 다른 군집 알고리즘

- 병합 군집: 밑바닥부터 클러스터 계층 구성
- BIRCH: 대규모 소특성 
- 평균-이동: 클러스터를 원으로 그림, 소규모
- 유사도 전파: 투표 방식, 소규모
- 스펙트럼 군집: 소규모, 동등한 클러스터 크기

### 9.2 가우시안 혼합 (GMM)

: 타원형 클러스터, 기댓값-최대화(EM) 알고리즘 사용



```
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=3, n_init=10)
```




In [ ]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10, random_state=42)

In [ ]:
gm.weights_
gm.means_
gm.covariances_

In [ ]:
gm.converged_ #수렴 여부
gm.n_iter_ #반복 횟수

In [ ]:
gm.predict(X)
gm.predict_proba(X)

In [ ]:
# 생성 모델: 모델에서 새로운 샘플 생성 가능
X_new, y_new = gm.sample(6)
X_new

In [ ]:
# 로그 확률 밀도 함수 (PDF) 추정
gm.score_samples(X)

covariance_type 매개변수를 사용해 이 알고리즘이 찾을 공분산 행렬을 제한. 학습할 파라미터 개수를 제한하여 특성이나 클러스터가 많/적은 샘플의 최적의 솔루션 수렴

- "full"(기본값): 제약이 없음. 모든 군집이 어떤 크기의 타원도 될 수 있음
- "tied": 모든 군집이 동일하지만 어떤 타원도 가능(즉, 공분산 행렬을 공유)
- "spherical": 모든 군집이 원형이지만 지름은 다를 수 있음(즉, 분산이 다름)
- "diag": 군집은 어떤 크기의 타원도 될 수 있지만 타원은 축에 나란함(즉, 공분산 행렬이 대각 행렬)

#### 9.2.1 가우시안 혼합을 사용한 이상치 탐지

밀도가 낮은 지역에 있는 모든 샘플을 이상치로 보기 -> 사용할 밀도 임곗값 정하기



```
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4) #4%
anomalies = X[densities < density_threshold]
```


cf) 특이치 탐지

: 이상치가 없는 '깨끗한' 데이터셋에서 훈련


#### 9.2.2 클러스터 개수 선택하기

cf) k-means: 이너셔, 실루엣 점수

대신, BIC와 AIC를 최소화 하는 모델 찾기

$BIC=log(m)p-2log(\hat{L})$

$AIC = 2p-2log(\hat{L})$

- m: 샘플 수
- p: 학습 파라미터 수
- $\hat{L}$: likelihood function 최댓값



```
gm.bic(X)
gm.aic(X)
```



#### 9.2.3 베이즈 가우시안 혼합 모델

: 불필요한 클러스터의 가중치를 0으로 만들기 (최적의 클러스터 개수를 수동으로 찾지 않기)



```
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components#최적의 클러스터 개수보다 큰 값#, n_init=#)
bgm.fit(X)
bgm.weights_ #클러스터 갯수 확인
```

- 사전 확률: 잠재 변수 z 사전 지식에 대한 확률 분포
- 사전 믿음: wight_concentration_prior 매개변수

- ELBO 최대화
- BBSVI (블랙 박스 확률적 변분 추론)

In [ ]:
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=10, n_init=10, random_state=42)
bgm.fit(X)
np.round(bgm.weights_, 2)

array([0.12, 0.12, 0.13, 0.16, 0.12, 0.13, 0.14, 0.  , 0.08, 0.  ])

#### 9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘

- **PCA(그리고 ```inverse_transform()```메서드를 가진 다른 차원 축소 기법)**: 이상치 탐지 기법 
- **Fast-MCD**: 이상치 감지하여 데이터셋을 정체, 샘플이 하나의 가우시안 분포에서 생성되었다고 가정하여 분포에 어긋난 값은 이상치로 여김. 타원형 데이터에 용이
- **아이솔레이션 포레스트**: 고차원 데이터. 무작위로 성장한 결정 트리로 구성된 랜덤 포레스트 생성.랜덤한 임곗값을 골라 데이터를 둘로 나누어 다른 샘플과 격리될 때까지 반복
-**LOF**: 주어진 샘플 주위의 밀도와 이웃 주위의 밀도를 비교하여 이상치 탐지
- **one-class SVM**: 특이치 탐지. 원본 공간으로부터 고차원 공간 샘플 분리. 원본 공간에 새로운 샘플이 없다면 이상치로 간주.